<a href="https://colab.research.google.com/github/VedantDere0104/GANs/blob/main/CycleGan_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####

In [ ]:
'''dir = '/content/drive/MyDrive/'
zip_path = '/content/drive/MyDrive/horse2zebra.zip'
'''

"dir = '/content/drive/MyDrive/'\nzip_path = '/content/drive/MyDrive/horse2zebra.zip'\n"

In [ ]:
'''!unzip '/content/drive/MyDrive/horse2zebra.zip' -d '/content/drive/MyDrive/'
'''

"!unzip '/content/drive/MyDrive/horse2zebra.zip' -d '/content/drive/MyDrive/'\n"

In [ ]:
'''img_domain_1_test = '/content/drive/MyDrive/horse2zebra/testA/'
img_domain_2_test = '/content/drive/MyDrive/horse2zebra/testB/'
img_domain_1_train = '/content/drive/MyDrive/horse2zebra/trainA/'
img_domain_2_train = '/content/drive/MyDrive/horse2zebra/trainB/' 
'''

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot
from numpy.random import randint

In [ ]:
def load_images(paths , size = (256 , 256)):
  datalist = list()

  for filenames in tqdm(os.listdir(paths)):
    img = tf.keras.preprocessing.image.load_img(paths + filenames , target_size=size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    datalist.append(img)
  return np.asarray(datalist)

In [ ]:
domain_1_train = load_images(img_domain_1_train)
domain_2_train = load_images(img_domain_2_train)
domain_1_test = load_images(img_domain_1_test)
domain_2_test = load_images(img_domain_2_test)

KeyboardInterrupt: ignored

In [ ]:
filename = '/content/drive/MyDrive/horse2zebra/horses2zebras_train.npz'
'''np.savez_compressed(filename , domain_1_train , domain_2_train)'''

'np.savez_compressed(filename , domain_1_train , domain_2_train)'

In [ ]:
test_filename = '/content/drive/MyDrive/horse2zebra/horses2zebras_test.npz'
'''np.savez_compressed(test_filename , domain_1_test , domain_2_test )'''

'np.savez_compressed(test_filename , domain_1_test , domain_2_test )'

In [ ]:
data = np.load(filename)
data_A , data_B = data['arr_0'] , data['arr_1']

In [ ]:
def define_discriminator(input_image_shape):
  init = tf.keras.initializers.RandomNormal(stddev=0.02)
  in_src_img = tf.keras.layers.Input(input_shape=input_image_shape)
  ## C64
  d = tf.keras.layers.Conv2D(64 , (4 , 4) , strides = (2 , 2) , padding='same' , kernel_initializer=init)(in_src_img)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
  ## C128
  d = tf.keras.layers.Conv2D(128 , (4 , 4) , strides=(2 , 2) , padding = 'same' , kernel_initializer=init)(d)
  d = tf.keras.layers.BatchNormalization()(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
  ## C256
  d = tf.keras.layers.Conv2D(256 , (4 , 4) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(d)
  d = tf.keras.layers.BatchNormalization()(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
  ## C512
  d = tf.keras.layers.Conv2D(512 , (4 , 4) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(d)
  d = tf.keras.layers.BatchNormalization()(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
  ## Second Last
  d = tf.keras.layers.Conv2D(512 , (4 , 4) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(d)
  d = tf.keras.layers.BatchNormalization()(d)
  d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
  ## Patch Out
  patch_out = tf.keras.layers.Conv2D(1 , (4 , 4) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(d)
  model = tf.keras.Model(in_src_img , patch_out)
  model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002 , beta_1=0.5 ) , loss_weights=[0.5])
  return model

In [ ]:
def resnet_block(n_filters , input_layer):
  init = tf.keras.initializers.RandomNormal(stddev=0.02)
  g = tf.keras.layers.Conv2D(n_filters , (4 , 4) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(input_layer)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  g = tf.keras.layers.Conv2D(n_filters , (4 , 4)  , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.Concatenate()([g , input_layer])
  return g

In [ ]:
def define_generator(input_image_shape , n_resnet = 9):
  init = tf.keras.initializers.RandomNormal(stddev=0.02)
  in_src_img = tf.keras.layers.Input(shape=input_image_shape)
  ## C7-S1-64
  g = tf.keras.layers.Conv2D(64 , (7 , 7) , strides=(1 , 1) , padding='same' , kernel_initializer=init)(in_src_img)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  ## 128
  g = tf.keras.layers.Conv2D(128 , (3 , 3) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  ## 256
  g = tf.keras.layers.Conv2D(256 , (3 , 3) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  ## Resnet 256
  for _ in range(n_resnet):
    g = resnet_block(256 , g)
  ## U128
  g = tf.keras.layers.Conv2DTranspose(128 , (3 , 3) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  ## U64
  g = tf.keras.layers.Conv2DTranspose(64 , (3 , 3) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.BatchNormalization()(g)
  g = tf.keras.layers.ReLU()(g)
  ## C7S1-3
  g = tf.keras.layers.Conv2DTranspose(3 , (7 , 7) , strides=(2 , 2) , padding='same' , kernel_initializer=init)(g)
  g = tf.keras.layers.BatchNormalization()(g)
  out_img = tf.keras.layers.Activation('tanh')(g)
  model = tf.keras.Model(in_src_img , out_img)
  return model
  


In [ ]:
def define_composite_model(g_model_1 , d_model , g_model_2 , image_shape):
  g_model_1.trainable = True
  d_model.trainable = False
  g_model_2.trainable = False
  input_gen = tf.keras.layers.Input(shape=image_shape)
  gen_1_out = g_model_1(input_gen)
  output_d = d_model(gen_1_out)

  input_id = tf.keras.layers.Input(shape = image_shape)
  output_id = g_model_1(input_id)
  ## Forward Cycle
  output_f = g_model_2(gen_1_out)
  ## Backwad Cycle
  gen_2_out = g_model_2(input_id)
  output_b = g_model_1(gen_2_out)

  model = tf.keras.Model([input_gen , input_id] , [output_d , output_id , output_f , output_b])
  opt = tf.keras.optimizers.Adam(learning_rate=0.0002 , beta_1=0.5)
  model.compile(loss = ['mse' , 'mae' , 'mae' , 'mae'] , loss_weights = [1 , 5 , 10 , 10] , optimizer = opt)
  return model





In [ ]:
def load_real_samples(filename):
  data = np.load(filename)
  X1 , X2 = data['arr_0'] , data['arr_1']
  X1 = (X1 - 127.5) / 127.5
  X2 = (X2 - 127.5) / 127.5
  return [X1 , X2]

In [ ]:
def generate_real_samples(dataset , n_samples , patch_shape):
  ix = randint(0 , dataset.shape[0] , n_samples)
  X = dataset[ix]
  y = np.ones((n_samples , patch_shape , patch_shape , 1))
  return X , y

In [ ]:
def generate_fake_samples(g_model , dataset , patch_shape):
  X = g_model.predict(dataset)
  y = np.zeros((len(X) , patch_shape , patch_shape , 1))
  return X , y

In [ ]:
def save_model(step , g_model_A_to_B , g_model_B_to_A):
  filename_1 = 'g_model_A_to_B.h5'
  g_model_A_to_B.save(filename_1)
  filename_2 = 'g_model_B_to_A.h5'
  g_model_B_to_A.save(filename_2)
  print('>Saved: %s and %s' % (filename1, filename2))
  


In [ ]:
def summarize_performance(step , g_model , train_X , name , n_samples = 5):
  X_in , _ = generate_real_samples(train_X , n_samples , 0)
  X_out , _ = generate_fake_samples(g_model , X_in , 0)
  X_in = (X_in + 1) / 2.0
  X_out = (X_out + 1) / 2.0
  for i in range(n_samples):
    pyplot.subplot(2, n_samples, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(X_in[i])
# plot translated image
  for i in range(n_samples):
    pyplot.subplot(2, n_samples, 1 + n_samples + i)
    pyplot.axis('off')
    pyplot.imshow(X_out[i])
# save plot to file
  filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
  pyplot.savefig(filename1)
  pyplot.close()


In [ ]:
def update_image_pool(pool, images, max_size=50):
  selected = list()
  for image in images:
    if len(pool) < max_size:
      # stock the pool
      pool.append(image)
      selected.append(image)
    elif random() < 0.5:
      # use image, but don✬t add it to the pool
      selected.append(image)
    else:
  # replace an existing image and use replaced image
      ix = randint(0, len(pool))
      selected.append(pool[ix])
      pool[ix] = image
  return asarray(selected)


In [ ]:
def train(d_model_A , d_model_B , g_model_A_to_B , g_model_B_to_A , c_model_A_to_B , c_model_B_to_A , dataset):
  n_epochs , n_batch = 100 , 1
  n_patch = d_model_A.output_shape[1]
  train_A , train_B = dataset
  pool_A , pool_B = list() , list()
  batch_per_epoch = int(len(train_A)/n_batch)
  n_steps = batch_per_epoch * n_epochs
  
  for i in range(n_steps):
    X_real_A , y_real_A = generate_real_samples(train_A , n_batch , n_patch)
    X_real_B , y_real_B = generate_real_samples(train_B , n_batch , n_patch)

    X_fake_A , y_fake_A = generate_fake_samples(g_model_B_to_A , X_real_B , n_patch)
    X_fake_B , y_fake_B = generate_fake_samples(g_model_A_to_B , X_real_A , n_patch)
    X_fake_A = update_image_pool(pool_A , X_fake_A)
    X_fake_B = update_image_pool(pool_B , X_fake_B)

    g_loss_2 , _ , _ , _ , _ = c_model_B_to_A.train_on_batch([X_real_B , X_real_A] , [y_real_A , X_real_A , X_real_B , X_real_A])
    dB_loss1 = d_model_B.train_on_batch(X_real_B , y_real_B)
    dB_loss2 = d_model_B.train_on_batch(X_fake_B , y_fake_B)
    print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2,dB_loss1,dB_loss2, g_loss1,g_loss2))
    if (i+1) % (bat_per_epo * 1) == 0:
      # plot A->B translation
      summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
      # plot B->A translation
      summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
    if (i+1) % (bat_per_epo * 5) == 0:
      #  save the models
      save_models(i, g_model_AtoB, g_model_BtoA)
    



In [ ]:
dataset = load_real_samples(filename)

In [ ]:
print(dataset[0].shape , dataset[1].shape)
image_shape = dataset[0].shape[1:]


(1067, 256, 256, 3) (1334, 256, 256, 3)


In [ ]:
g_model_A_to_B = define_generator(image_shape)
g_model_B_to_A = define_generator(image_shape)
d_model_A = define_discriminator(image_shape)
d_model_B = define_discriminator(image_shape)
c_model_A_to_B = define_composite_model(g_model_A_to_B , d_model_B , g_model_B_to_A , image_shape)
c_model_B_to_A = define_composite_model(g_model_B_to_A , d_model_A , g_model_A_to_B , image_shape)
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)
